In [1]:
from dataclasses import dataclass
import os
import sys

In [2]:
%%capture
import mercury as mr
import pandas as pd
from IPython.display import display

from lucupy.minimodel.constraints import CloudCover, ImageQuality
from lucupy.minimodel.semester import SemesterHalf
from lucupy.minimodel.site import Site
from lucupy.observatory.abstract import ObservatoryProperties
from lucupy.observatory.gemini import GeminiProperties

from definitions import ROOT_DIR
from scheduler.core.builder.blueprint import CollectorBlueprint, OptimizerBlueprint
from scheduler.core.builder.builder import SchedulerBuilder
from scheduler.core.components.collector import *
from scheduler.core.output import (print_collector_info, plans_table, pickle_plans, pickle_selection)
from scheduler.core.programprovider.ocs import read_ocs_zipfile, OcsProgramProvider
from scheduler.core.statscalculator import StatCalculator

In [3]:
# Add the parent directory to the Python search path.
cwd = os.getcwd().replace('/demo', '')
sys.path.insert(0, cwd)

# Application parameters.
app = mr.App(title="Scheduler Validation Test",
        description="Try the Validation mode for the Scheduler",
        show_code=False,
        show_prompt=True,
        continuous_update=False,
        static_notebook=False,
        show_sidebar=True,
        full_screen=True,
        allow_download=True)

mercury.App

In [4]:
out_dir = mr.OutputDir()
print(f"Output directory path: {out_dir.path}")

mercury.OutputDir

Output directory path: .


In [2]:
# Set up the UI.
start_date = mr.Text(label='Start Date', value='2018-10-01')
start_time = mr.Text(label='Start Time (UTC)', value='08:00:00')  
end_date = mr.Text(label='End Date', value='2018-10-03')
end_time = mr.Text(label='End Time (UTC)', value='08:00:00')

site_choices = [site.name for site in Site]
site_selection = mr.MultiSelect(label="Sites",
                                choices=site_choices,
                                value=site_choices)

cc_choices = [cc.name for cc in CloudCover]
cc_selection = mr.Select(label='Cloud Cover',
                         choices=cc_choices,
                         value=CloudCover.CC50.name)

iq_choices = [iq.name for iq in ImageQuality]
iq_selection = mr.Select(label='Image Quality',
                         choices=iq_choices,
                         value=ImageQuality.IQ70.name)


@dataclass(frozen=True)
class Params:
    start: Time
    end: Time
    num_nights_to_schedule: int
    sites: FrozenSet[Site]
    cc: CloudCover
    iq: ImageQuality
       
def extract_params() -> Params:
    start_string = f'{start_date.value} {start_time.value}'
    try:
        start = Time(start_string, format='iso', scale='utc')
    except Exception as e:
        print(f'Could not parse the start string "{start_string}".')
        raise mr.StopExecution
        
    end_string = f'{end_date.value} {end_time.value}'
    try:
        end = Time(end_string, format='iso', scale='utc')
    except Exception as e:
        print(f'Could not parse the end string "{end_string}".')
        raise mr.StopExecution
        
    if end <= start:
        print(f'Start of run "{start_string}" must be before end "{end_string}".')
        raise mr.StopExecution
    num_nights_to_schedule = int(round(end.jd - start.jd)) + 1
    
    sites = frozenset(Site[s] for s in site_selection.value)
    if not sites:
        print('No sites specified.')
        raise mr.StopExecution
    
    try:
        cc = CloudCover[cc_selection.value]
    except KeyError:
        print(f'Illegal CloudCover: "{cc_selection.value}"')
        raise mr.StopExecution
    
    try:
        iq = ImageQuality[iq_selection.value]
    except KeyError:
        print(f'Illegal ImageQuality: "{iq_selection.value}"')
        raise mr.StopExecution
    
    return Params(start=start,
                  end=end,
                  num_nights_to_schedule=num_nights_to_schedule,
                  sites=sites,
                  cc=cc,
                  iq=iq)


SyntaxError: f-string: empty expression not allowed (934748621.py, line 60)

## SCHEDULER 

When the Scheduler is running, the UI will be frozen until the results are displayed.
When the run button is enabled, the Scheduler may be invoked.
The default values provided are the ones used during internal testing.

In [6]:
# Validate the UI data.
params = extract_params()

print('Parameters parsed: Scheduler execution running...\n')

# If validation is successful, we run the Scheduler.
ObservatoryProperties.set_properties(GeminiProperties)

# Read in a list of JSON data
programs = read_ocs_zipfile(os.path.join(ROOT_DIR, 'scheduler', 'data', '2018B_program_samples.zip'))

# Create the Collector and load the programs.
collector_blueprint = CollectorBlueprint(
    ['SCIENCE', 'PROGCAL', 'PARTNERCAL'],
    ['Q', 'LP', 'FT', 'DD'],
    1.0
)

collector = SchedulerBuilder.build_collector(
    start=params.start,
    end=params.end,
    sites=params.sites,
    semesters=frozenset([Semester(2018, SemesterHalf.B)]),
    blueprint=collector_blueprint
)

# Create the Collector and load the programs.
collector.load_programs(program_provider_class=OcsProgramProvider,
                        data=programs)

print_collector_info(collector, samples=10)

selector = SchedulerBuilder.build_selector(collector,
                                           num_nights_to_schedule=params.num_nights_to_schedule,
                                           default_cc=params.cc,
                                           default_iq=params.iq)

# Prepare the optimizer.
optimizer_blueprint = OptimizerBlueprint(
    "GreedyMax"
)
optimizer = SchedulerBuilder.build_optimizer(
    blueprint=optimizer_blueprint
)

# The total nights for which visibility calculations have been done.
total_nights = len(collector.time_grid)

# Create the overall plans by night.
overall_plans = {}

for night_idx in range(selector.num_nights_to_schedule):
    # Get the night indices for which we are selecting.
    # TODO: We will want scores for nights to look ahead for greedy optimization.
    # TODO: For now, we use the entire period for which visibility calculations have been done.
    # night_indices = range(night_idx, total_nights)
    night_indices = np.array([night_idx])
    # selection = selector.select(night_indices=np.array([0, 1, 2])
    selection = selector.select(night_indices=night_indices)
    pickle_selection(selection, out_dir.path, night_idx)
    # Run the optimizer to get the plans for the first night in the selection.
    # mr.Md(f'### GreedyMax Output for night {night_idx}')
    plans = optimizer.schedule(selection)
    night_plans = plans[0]

    # Store the plans in the overall_plans array for that night.
    # TODO: This might be an issue. We may need to index nights (plans) in optimizer by night_idx.
    overall_plans[night_idx] = night_plans

    # Perform the time accounting on the plans.
    collector.time_accounting(night_plans)


overall_plans = [p for _, p in sorted(overall_plans.items())]
plan_summary = StatCalculator.calculate_plans_stats(overall_plans,collector)
# print_plans(overall_plans)
night_tables = plans_table(overall_plans)
pickle_plans(night_tables, out_dir.path, params.start, params.end)
for n_idx, table_per_site in enumerate(night_tables):
    mr.Md(f"## Night Idx: {n_idx+1} ")
    for site in params.sites:
        night_stats = overall_plans[n_idx][site].night_stats
        mr.Md(f"### Site: {site.value[0]} ")
        display(table_per_site[site])
        mr.Md(f'Time loss: {night_stats.timeloss}')
        mr.Md(f'Plan_Score: {night_stats.plan_score}')
        mr.Md(f'Number of ToOs: {night_stats.n_toos}')
        mr.Md(f'Schedule Observations by band')
        for key, value in night_stats.completion_fraction.items():
            mr.Md(f'Band {key:<10}{value}')
    mr.Md('---')            
mr.Md('## Program Completion')
df_pcompl = pd.DataFrame(plan_summary).T.rename(columns={0: "% Completion", 1: "Score"})
display(df_pcompl)

print('DONE')


mercury.Text

mercury.Text

mercury.Text

mercury.Text

mercury.MultiSelect

...

Band 1		6
Band 2		4
Band 3		0
Band 4		0


---

## Program Completion

,% Completion,Score
GN-2018B-Q-230,100.0%,11961.177467
GN-2018B-Q-104,62.5%,7415.627402
GN-2018B-Q-903,0.0%,17.87151
GN-2018B-Q-101,56.4%,4828.102079
GN-2018B-Q-133,18.2%,181.032429
GS-2018B-Q-223,91.7%,21559.234535
GS-2018B-Q-104,50.0%,487.995793
GS-2018B-Q-105,23.5%,289.712623
GS-2018B-Q-102,0.0%,643.953526
GN-2018B-Q-108,0.0%,23.602973


DONE
